In [62]:
import pandas as pd
import keras


In [134]:
df = pd.read_csv('../data/data_phdosado.csv', index_col=1)
df = df.drop(columns=['ID', 'Real_RD_PV_BaumeCal'])

In [135]:
list(df.columns)

['Real_RD_PV_pHDosado',
 'Real_RD_MV_ValvulaCalpHDosado',
 'Real_RD_PV_VazaoDosado',
 'Real_RD_ST_LimpezaPHmetro',
 'Real_RD_PV_PressaoLinhaCal',
 'Real_RD_MV_ValvulaCalpHDosado-2',
 'Real_RD_MV_PressaoLinhaCal',
 'Real_RD_PV_NivelTqCal',
 'Real_RD_PV_NivelTqDosado']

In [136]:
df.head()


,Real_RD_PV_pHDosado,Real_RD_MV_ValvulaCalpHDosado,Real_RD_PV_VazaoDosado,Real_RD_ST_LimpezaPHmetro,Real_RD_PV_PressaoLinhaCal,Real_RD_MV_ValvulaCalpHDosado-2,Real_RD_MV_PressaoLinhaCal,Real_RD_PV_NivelTqCal,Real_RD_PV_NivelTqDosado
timezone,,,,,,,,,
2024-07-13 14:28:22,7.1260,37.128124,899.625,0.0,0.798437,37.128124,74.546875,98.31250,73.84375
2024-07-13 14:28:25,7.1225,37.178123,899.625,0.0,0.798437,37.178123,74.578125,98.34375,73.87500
2024-07-13 14:28:28,7.1225,37.187500,899.625,0.0,0.796875,37.187500,74.559375,98.40625,73.87500
2024-07-13 14:28:31,7.1260,37.331249,899.625,0.0,0.798437,37.331249,74.596875,98.50000,73.84375
2024-07-13 14:28:34,7.1225,37.218750,900.375,0.0,0.798437,37.218750,74.562500,98.25000,73.87500


## Pré-processamento

In [137]:
minute = 20 # Observações por minuto
hour = minute * 60
day = hour * 24

In [138]:
# Seleciona apenas os primeiros dez dias
df = df[:(day * 10)]

In [139]:
# Normalização
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)
normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

In [ ]:
# Porcentagem usada para treinar o modelo
split_fraction = 0.715
train_split = int(split_fraction * int(df.shape[0]))

step = 1

# Usaremos os dados da última hora para prever 5 minutos
past = hour
future = minute * 5

learning_rate = 0.001
batch_size = 256
epochs = 10

In [141]:
train_data = normalized_df.loc[0 : train_split - 1]
val_data = normalized_df.loc[train_split:]

### Dataset de treinamento

In [142]:
# Dataset de treinamento
start = past + future
end = start + train_split

x_train = train_data.values
# y_train = normalized_df['Real_RD_PV_pHDosado'][:train_split - 1].values
y_train = normalized_df['Real_RD_PV_pHDosado'][start:end]

sequence_length = int(past / step)

In [143]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

In [144]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

x_train shape: (205920, 9)
y_train shape: (205920,)


In [145]:
for batch in dataset_train.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (256, 1200, 9)
Target shape: (256,)


### Dataset de validação

In [146]:
# Dataset de validação
x_end = len(val_data) - past - future
label_start = train_split + past + future

x_val = val_data.iloc[:x_end].values
y_val = normalized_df['Real_RD_PV_pHDosado'][label_start:].values

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

In [147]:
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")

x_val shape: (80780, 9)
y_val shape: (80780,)


# Treinamento

In [148]:
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = keras.layers.LSTM(32)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1200, 9)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,409 (21.13 KB)

 Trainable params: 5,409 (21.13 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
path_checkpoint = "model_checkpoint.weights.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

history = model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val,
    callbacks=[es_callback, modelckpt_callback],
)

# Referências

- [Timeseries forecasting for weather prediction](https://keras.io/examples/timeseries/timeseries_weather_forecasting/)